<a href="https://colab.research.google.com/github/viswanadhgompa/AQI/blob/main/Ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Step 1: Load Data
df = pd.read_csv("data.csv", encoding="ISO-8859-1")

# Step 2: Handle Missing Values
df.fillna(method='ffill', inplace=True)  # Forward fill missing values

# Step 3: Feature Selection & Scaling
features = ['pm2_5', 'rspm', 'no2']
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

# Step 4: Convert Data into Time-Series Format
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 10  # Sequence length
X, y = create_sequences(df[features].values, seq_length)

# Step 5: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Build LSTM Model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(seq_length, len(features))),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(len(features))
])

model.compile(optimizer='adam', loss='mse')

# Step 7: Train Model
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test), verbose=1)

# Step 8: Predictions
y_pred = model.predict(X_test)
y_pred_rescaled = scaler.inverse_transform(y_pred)
y_test_rescaled = scaler.inverse_transform(y_test)

# Step 9: Visualizations
plt.figure(figsize=(12, 6))
plt.plot(y_test_rescaled[:, 0], label='Actual PM2.5')
plt.plot(y_pred_rescaled[:, 0], label='Predicted PM2.5', linestyle='dashed')
plt.legend()
plt.title('Actual vs Predicted PM2.5 Levels')
plt.show()

# Step 10: Identify High-Risk States
state_avg_pollution = df.groupby('state')['pm2_5'].mean()
plt.figure(figsize=(10, 6))
sns.barplot(x=state_avg_pollution.index, y=state_avg_pollution.values, palette='Reds')
plt.xticks(rotation=90)
plt.title('Average PM2.5 Levels by Country')
plt.show()

# Further visualizations can be added based on analysis!


<ipython-input-2-90bc98f967b1>:12: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data.csv", encoding="ISO-8859-1")
<ipython-input-2-90bc98f967b1>:15: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)  # Forward fill missing values


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


21780/21787 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: nan

In [ ]:
print(df.columns)


Index(['stn_code', 'sampling_date', 'state', 'location', 'agency', 'type',
       'so2', 'no2', 'rspm', 'spm', 'location_monitoring_station', 'pm2_5',
       'date'],
      dtype='object')
